# Контурное управление со сглаживанием на окружностях

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Рассмотрим сглаживание траектории с помощью окружностей.
При использовании окружности мы можем гарантировать сохранение постоянной скорости.

Зададим начальную, поомежуточную и конечную точки, укажем радиус зоны сглаживания.

In [ ]:
s = Vector(2, -4, 0)
i = Vector(2, 3, 5)
e = Vector(-3, 3, 0)
r = 2

In [ ]:
d1 = i + -1 * s
d2 = e + -1 * i
n = d1.cross(d2).normalized()
nd = i + n

In [ ]:
p1 = d1 + -1 * r * d1.normalized()
nn = n.cross(d1).normalized()
nnd = i + nn

In [ ]:
a = d1.angle_to(-1 * d2, n)
l = r / np.cos(a / 2)
rr = r * np.tan(a / 2)
c = i + Quaternion.from_angle_axis(a / 2, n) * (-l * d1.normalized())
b = np.pi + a

def fillet(t):
    pos = c + Quaternion.from_angle_axis(-b / 2 + b * t, n) * (-1 * rr * (i + -1 * c).normalized())
    return (
        pos.x,
        pos.y,
        pos.z
    )

In [ ]:
l1 = d1.magnitude() - r
l2 = np.abs(b * rr)
l3 = d2.magnitude() - r

def path(t):
    total = l1 + l2 + l3
    position = t * total
    if position < l1:
        progress = position / l1
        return s + progress * l1 * d1.normalized()
    if position < l1 + l2:
        progress = (position - l1) / l2
        f = fillet(progress)
        return Vector(f[0], f[1], f[2])
    if position < l1 + l2 + l3:
        progress = (position - l1 - l2) / l3
        return i + (r + (progress * l3)) * d2.normalized()
    return Vector.zero()

In [ ]:
fig, ax = graphics.figure(6)
ax.plot([s.x, i.x, e.x], [s.y, i.y, e.y], [s.z, i.z, e.z], label="path")
ax.plot([i.x, nd.x], [i.y, nd.y], [i.z, nd.z], label="path plane normal")
ax.plot([i.x, nnd.x], [i.y, nnd.y], [i.z, nnd.z], label="normal normal")
ax.plot([i.x, c.x], [i.y, c.y], [i.z, c.z], label="center connector")

l, = ax.plot([], [], [], label="motion")

t = np.arange(0, 1, 0.01)
f = fillet(t)
ax.plot(f[0], f[1], f[2], label="fillet")

total = 100

def animate(frame):
    t = frame / total
    p = path(t)
    l.set_data_3d([0, p.x], [0, p.y], [0, p.z])

fig.legend()

animate(50)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

Как видно, скорость $V$ остается постоянной на всей траектории.

In [ ]:
v_path = np.vectorize(path)
step = 0.001
t = np.arange(0, 1, step)
pos = v_path(t)
x = []
y = []
z = []
for p in pos:
    x += [p.x]
    y += [p.y]
    z += [p.z]
vx = np.diff(x) / step
vy = np.diff(y) / step
vz = np.diff(z) / step
v = (vx ** 2 + vy ** 2 + vz ** 2) ** 0.5

acx = np.diff(vx) / step
acy = np.diff(vy) / step
acz = np.diff(vz) / step

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, x, color="#c00000", label="$x$")
ax.plot(t, y, color="#00c000", label="$y$")
ax.plot(t, z, color="#0000c0", label="$z$")
ax.legend()

ava = fig.add_subplot(2, 2, 2)
ava.plot(t[:-1], v, label="$v$")
ava.set_ylim((0, np.max(v) + 1))
ava.legend()

av = fig.add_subplot(2, 2, 3)
av.plot(t[:-1], vx, color="#c00000", label="$V_x$")
av.plot(t[:-1], vy, color="#00c000", label="$V_y$")
av.plot(t[:-1], vz, color="#0000c0", label="$V_z$")
av.legend()

aa = fig.add_subplot(2, 2, 4)
aa.plot(t[:-2], acx, color="#c00000", label="$a_x$")
aa.plot(t[:-2], acy, color="#00c000", label="$a_y$")
aa.plot(t[:-2], acz, color="#0000c0", label="$a_z$")
aa.legend()

fig.show()